In [27]:
print("Hello!")

Hello!


In [28]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [29]:
PINECONE_API_KEY = "************************"
PINECONE_API_ENV = "*******"

In [30]:
# Extract the data from the pdf book
def load_data(data):
    loader = DirectoryLoader(data,
    glob="*.pdf",
    loader_cls=PyPDFLoader)

    documents = loader.load()
    return documents
    

In [31]:
extracted_pdf = load_data("Data/")
print("Data Extracted")

Data Extracted


In [32]:
#create text Chunks
def text_split(extracted_pdf):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,
                                   chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_pdf)
    return text_chunks 

In [33]:
text_chunks = text_split(extracted_pdf)
print("chunks created")
print("The number of chunks is:",len(text_chunks))

chunks created
The number of chunks is: 7020


In [34]:
#download embedding model
def download_huggingfaceembedding():
    embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embedding

In [35]:
embedding = download_huggingfaceembedding()

In [36]:
embedding

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

In [37]:
query_result = embedding.embed_query("Hello World!")
print("Length:", len(query_result))
query_result

Length: 384


[-0.020386839285492897,
 0.025280786678195,
 -0.0005662619369104505,
 0.011615419760346413,
 -0.037988368421792984,
 -0.11998126655817032,
 0.04170948639512062,
 -0.020857201889157295,
 -0.05900674685835838,
 0.02423257753252983,
 0.06212017685174942,
 0.06767988204956055,
 0.03310022130608559,
 -0.010369356721639633,
 -0.031215675175189972,
 -0.03273328021168709,
 -0.002111797221004963,
 0.00926193967461586,
 -0.12476454675197601,
 0.011236845515668392,
 0.03904542326927185,
 0.05440244451165199,
 -0.002825475065037608,
 0.044556330889463425,
 -0.08542019873857498,
 -0.02287374623119831,
 0.039140600711107254,
 0.036046914756298065,
 -0.03212681785225868,
 -0.0642586499452591,
 0.05812908709049225,
 0.046690843999385834,
 0.0806155577301979,
 -0.007734252605587244,
 -0.022083155810832977,
 0.06713150441646576,
 -0.045041438192129135,
 -0.1021212711930275,
 0.0012644092785194516,
 0.046801935881376266,
 0.026395926252007484,
 -0.06990955024957657,
 -0.044533487409353256,
 -0.0069019282

In [39]:
#Initializing the Pinecone
pinecone.init(api_key=PINECONE_API_KEY,
              environment=PINECONE_API_ENV)

index_name="medical-chatbot"

#Creating Embeddings for Each of The Text Chunks & storing
docsearch=Pinecone.from_texts([t.page_content for t in text_chunks], embedding, index_name=index_name)

In [41]:
# After having my index, we can load it like this - 
docsearch = Pinecone.from_existing_index(index_name=index_name, embedding=embedding)

query = "What are Allegies?"

docs = docsearch.similarity_search(query=query, k=3)

print("Result:", docs)

Result: [Document(page_content='and the offending substanceis called an allergen.', metadata={}), Document(page_content='When thisoccurs, an allergy develops against the offending sub-stance (an allergen.)', metadata={}), Document(page_content='generally follow a basic method. The allergen is boundto a solid support, either in the form of a cellulosesponge, microtiter plate, or paper disk. The patient’sserum is prepared from a blood sample and is incubatedwith the solid phase. If allergen specific IgE antibodiesare present, they will bind to the solid phase and beretained there when the rest of the serum is washedaway. Next, an labeled antibody against the IgE is addedand will bind to any IgE on the solid phase. The excessis washed', metadata={})]


In [42]:
prompt_template = """
Use the following piece of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to generate any random answer from your own

Context:{context}
Question:{question}

Only return the helpful answer and nothing else
helpful answer:
"""

In [43]:
PROMPT = PromptTemplate(template=prompt_template, input_variables=["context","question"])
chain_type_kwargs = {"prompt":PROMPT}

In [44]:
llm = CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                    model_type="llama",
                    config={"max_new_tokens":512,
                            'temperature':0.8})

In [45]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever= docsearch.as_retriever(search_kwargs={"k":2}),
    return_source_documents = True,
    chain_type_kwargs=chain_type_kwargs)

In [46]:
while True:
    user_input = input(f"Input Prompt: ")
    result = qa({"query":user_input})
    print("Response:", result["result"])

Response: Acne is a common skin disease characterized by pimples on the face, chest, and back. It occurs when the pores of the skin become clogged with oil, dead skin cells, and bacteria.
